# Running a Polymer Welding Workflow with Hoomd-Organics

## Overview:
In this tutorial, we'll show a workflow that involves multiple steps of initializing a system and running simulations. In this case, part of initializing a system invovles running a simulation where the final structure of the simulaton serves as the input for the next round of system building and simulation.

**Specifically, the goal of this workflow is to model the fusion joining (i.e. welding) of two separate pieces of bulk polymer melt**. This requires that we have an equilibrated polymer melt system with at least 1 flat surface (i.e. where the periodic boundaries are eliminated). For the purposes of this tutorial, we will refer to this as a "slab". Once a slab structure has been created, we will create an interface system where two slabs are joined together, and run a fusion welding simulation. Finally, the welded system will be tested using a tensile test simulation.

In summary, this tutorial will go through the following workflow:

1. Initialize a new polymer system using poly(ethylene)
2. Run a simulation which yields a poly(ethylene) slab
3. Build a system that is an interface between 2 poly(ethylene) slabs.
4. Run a fusion welding simulation
5. Run a tensile test simulation

The first step utilizes the same base classes dicussed in the introduction tutorial where we pull from `hoomd_organics.library.polymers` and `hoomd_organics.base.system` to build up poly(ethylen) chains and create an initial system. However, the rest of the workflow will use `hoomd_organics.modules.welding` and `hoomd_organics.library.simulations.tensile`.  These two modules inheret from the base `hoomd_organics.base.Simulation` class and contain additional features geared towards this specific workflow.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from hoomd_organics.library import PolyEthylene, OPLS_AA
from hoomd_organics import Pack
from hoomd_organics.modules.welding import SlabSimulation, Interface, WeldSimulation

from cmeutils.visualize import FresnelGSD
import gsd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import unyt as u

## Step 1: Creating a "Slab"
In the next cell, we will initialize a new system with 30 chains of poly(ethylene) with each chain containing 12 repeat units. We'll use the `Pack` class from `hoomd_organics.base.system`.

**Note:**
1. For the sake of speeding up the simulation, hydrogen atoms will be removed (i.e. we'll run a "united atom" simulation) and charges won't be included in the simulation forcefield.

In [ ]:
molecule = PolyEthylene(num_mols=30, lengths=12)

system = Pack(
    molecules=molecule,
    force_field=OPLS_AA(),
    density=1.1,
    r_cut=2.5,
    auto_scale=True,
    remove_charges=True,
    remove_hydrogens=True
)

#### `hoomd_organics.modules.welding.SlabSimulation`

In this cell, you'll notice we are not using `hoomd_organics.base.Simulation` but instead using `SlabSimulation`. This is a simulation class that inherits from `hoomd_organics.base.Simulation` but adds two additional features:

1. There is a parameter called `interface_axis` which determines which primary box face will be kept flat. This is required to create a interface system afterward.
2. hoomd wall forces (`hoomd.md.external.wall.LJ`) are how we can elimiate one of the periodic boundaries, and create a flat surface. These forces are automatically added to the overall forcefield, and set up depending on the value used for `interface_axis`

**Note:**  

Since we are running multiple simulations we'll utilize python pickeling in order to re-use the Hoomd forcefield. There is a built-in method in the `Simulation` class which makes this step easier

In [ ]:
sim = SlabSimulation.from_system(
    system=system,
    interface_axis=(1,0,0),
    gsd_file_name="slab_x_interface.gsd",
)

Here, we can look at the complete set of forces that make up the forcefield. You'll notice that in addition to the typical forces of bonds, angles, dihedrals and pairs, there is also a `hoomd.md.external.wall.LJ` force as we would expect.

In [ ]:
sim.forces

If we look at the details of the wall force we will notice two features:
1. Two walls are created where the origin for each is created along opposite ends of the x-axis of the simulation volume.
2. The forces of the walls point inward along the same axis.

In [ ]:
for wall in sim.forces[-1].walls:
    print(wall)
print()
print("Simulation volume:", sim.box_lengths_reduced)

Similar to the introduction tutorial, we will use the `run_update_volume` method to reach a desired bulk density of $1.2 \dfrac{g}{cm^3}$. It is important we maintain the walls while the volume is updated. `hoomd_organics.base.Simulation` checks for the presence of wall forces when using `run_update_volume` and ensures the wall positions updated along with the box.

In [ ]:
sim.run_update_volume(final_density=1.2*u.g/(u.cm**3), n_steps=5e4, kT=5.0, period=100, tau_kt=0.001)

Now that we reached the target bulk volume, we will run a `NVT` simulation using `run_NVT`. Normally, this step will be ran long enough to ensure the system reaches equilibration, but for the sake of this tutorial we'll run for a small number of steps.

In [ ]:
sim.run_NVT(kT=5.0, n_steps=2e4, tau_kt=0.001)

We can use the `pickle_forcefield` method to save and re-use the hoomd forces generated by the `System` class. This saves time when running multiple simulations that use the same set of forces. First, before calling this method, we will remove the wall forces.

In [ ]:
sim.remove_walls(wall_axis=(1,0,0))
sim.pickle_forcefield()

#### Visualizing the polyethylene slab:

We can use a util from `cmeutils` that utilizes the [**Fresnel**](https://fresnel.readthedocs.io/en/v0.13.5/) python package under the hood to let us view the simulation trajectory in this notebook.

In this case, we are looking down the y-axis of the simulation volume, with the x-axis running left to right. This is the final frame of the simulation trajectory.

In [ ]:
sim_viewer = FresnelGSD(gsd_file="slab_x_interface.gsd", view_axis=(0, 1, 0), frame=-1)
sim_viewer.view()

You can see that particles are not passing through the simulation volume in the x-direction (left/right). In fact, we can see a small amount of space within the simulation volume along that direction where particles can't access, this is because of the repulsive force caused by the wall.

Still, we can better visualize this using the unwrapped particle positions.

**Note**
We'll adjust the height paramter to zoom out our view since the particle positions are outside the simulation volume

In [ ]:
sim_viewer.unwrap_positions = True
sim_viewer.height += 3
sim_viewer.view()

We can change our view direction to look down the interface axis. We can see particles crossing the periodic boundaries in both the y direction (left/right) and z direction(up/down).

In [ ]:
sim_viewer.view_axis = (1,0,0)
sim_viewer.view()

## Step 2: Creating an interface from the slab

Now that the slab simulation is finished, we will use the final structure as the input to create an interface system. This system initialization step differs from those in `hoomd_organics.base.system` where the inputs are molecules, number of molecules, forcefield, etc. The welding module has its own class (`hoomd_organics.modules.welding.Interface`) desiged to create interfaces from slab simulation GSD files. The `Interface` class creates a new `gsd.hoomd.Snapshot` where the slab system is duplicated and translated along the interface axis.

If we look at the details of the new system we'll notice a couple things:
1. The number of particles has doubled, from 720 to 1440
2. The box lengths along the interface axis are doubled while the other two box lengths remain the same

In [ ]:
interface = Interface(gsd_file="slab_x_interface.gsd", interface_axis=(1, 0, 0), gap=0.05)

print(type(interface.hoomd_snapshot))
print("Slab number of particles:", system.n_particles)
print("Interface number of particles:", interface.hoomd_snapshot.particles.N)
print()
print("Slab box lengths:", sim.box_lengths_reduced)
print("Interface box lengths:", interface.hoomd_snapshot.configuration.box[:3])

## Step 3: Running a welding simulation



The hoomd forcefield was initially created in the `hoomd_organics.base.System` class. However, since the interface system is not initalized in the same way, we can reuse the hoomd forces we saved in a previous cell (by using `Simulation.pickle_forcefield()`) and pass it directly in this simulation instance.

In [ ]:
# Open and load the forcefield picke file
with open("forcefield.pickle", "rb") as f:
    hoomd_forces = pickle.load(f)

# Let's see what is stored in this pickle file
for force in hoomd_forces:
    print(force)

We'll initialize the `WeldSimulation` class in a similar way to initialzing any simulation in `hoomd_organics`.

Again, this particular simulation class has a `interface_axis` parameter that we will set depending on the interface axis used in the slab simulation.

For the `initial_state`, we'll call the `hoomd_snapshot` attribute from the `Interface` system created previously. For the `forcefield` parameter we'll pass in the list of hoomd force objects we loaded with `pickle`

In [ ]:
weld_sim = WeldSimulation(
    initial_state=interface.hoomd_snapshot,
    forcefield=hoomd_forces,
    interface_axis=(1, 0, 0),
    gsd_file_name="weld.gsd",
    log_file_name="weld_log.txt",
    log_write_freq=500,
    dt=0.0003
)

Similar to `SlabSimulation` the `WeldSimulaton` class also adds walls along the interface axis. This is because we only want diffusion and mixing to occur along one direction of the interface. We can see that a `hoomd.md.external.wall.LJ` object now exists in `weld_sim.forces` and the wall force details are aligned with the box geometry and `interface_axis` parameter.

In [ ]:
weld_sim.forces

In [ ]:
for wall in weld_sim.forces[-1].walls:
    print(wall)
print()
print("Simulation volume:", weld_sim.box_lengths_reduced)

#### Running the weld simulation
This process of polymer chain diffusion across the interface is slow, so for the purposes of this tutorial we likely won't be able to run long enough to see significant diffusion and entanglement across the interface.

**Note:**  
Our system size has doubled compared to the slab simulation, so the TPS (time steps per second) has significantly decreased.
This should have a TPS around 300, so 20,000 steps will take about 1 minute to run. You can change the value for `n_steps` accordingly.  

To achieve as much diffusion as possible in a short amount of time, we are also using a very high temperature.


In [ ]:
weld_sim.run_NVT(kT=10.0, n_steps=7e4, tau_kt=0.001)
cooling_ramp = weld_sim.temperature_ramp(n_steps=2e4, kT_start=10.0, kT_finish=1.0)
weld_sim.run_NVT(kT=cooling_ramp, n_steps=2e4, tau_kt=0.001)
weld_sim.save_restart_gsd("weld_restart.gsd")

#### Viewing the interface simulation before and after welding

We can load the GSD file output by `weld_sim` and view the first and last frames.
Here, we set the particle colors based on their starting position relative to the interface.
In the first frame, we can see the interface system now consists of two slabs, with a small gap between them along the interface axis.

In [ ]:
sim_viewer = FresnelGSD(gsd_file="weld.gsd", view_axis=(0, 1, 0), frame=0, height=12)
weld_colors = np.zeros_like(sim_viewer.positions)
weld_colors[:weld_colors.shape[0]//2 + 1] = np.array([0.5, 0.25, 0.5])
weld_colors[weld_colors.shape[0]//2 + 1:] = np.array([0.5, 0.1, 0.1])
sim_viewer.colors = weld_colors
sim_viewer.view(width=500, height=500)

After running for a short amount of time, we can see the gap at the interface is gone.

In [ ]:
sim_viewer.frame = -1
sim_viewer.height = 12
sim_viewer.view_axis = (0, 1, 0)
sim_viewer.view(width=500, height=500)

While we can't run long enough in this tutorial for significant chain diffusion to occur, it might still be interesting to look at the change in the Lennard Jones pair energy as the two slabs begin to adhere together. In the cell below, we plot the per-particle pair energy vs time step.

In [ ]:
sim_data = np.genfromtxt("weld_log.txt", names=True)
lj_energy = sim_data["mdpairLJenergy"]
time_step = sim_data["hoomd_organicsmodulesweldingWeldSimulationtimestep"]
plt.plot(time_step, lj_energy/interface.hoomd_snapshot.particles.N, linewidth=3)
plt.ylabel("$\dfrac{\epsilon}{N}$", fontsize=15)
plt.xlabel("Time step", fontsize=15)

# Step 4: Running a tensile test simulation

In [ ]:
from hoomd_organics.library.simulations.tensile import Tensile

In [ ]:
# Open and load the forcefield picke file
with open("forcefield.pickle", "rb") as f:
    hoomd_forces = pickle.load(f)
    
tensile_sim = Tensile(
    initial_state="weld_restart.gsd",
    forcefield=hoomd_forces,
    tensile_axis=(1,0,0),
    gsd_file_name="tensile.gsd",
    gsd_write_freq=1000,
    log_file_name="tensile_log.txt",
    log_write_freq=500
)

The `Tensile` simulation class has a new run method, called `run_tensile`. This run method has 2 important parameters.
1. strain: This determines 

In [ ]:
tensile_sim.run_tensile(n_steps=7e4, strain=0.15, period=500, kT=1.0)

The first frame of the tensile pulling simulation is the last frame we viewed above in the welding simulation.
Let's take a look at the system after the slabs were pulled apart.

In [ ]:
sim_viewer = FresnelGSD(gsd_file="tensile.gsd", view_axis=(0, 1, 0), frame=-1, height=16)
weld_colors = np.zeros_like(sim_viewer.positions)
weld_colors[:weld_colors.shape[0]//2 + 1] = np.array([0.5, 0.25, 0.5])
weld_colors[weld_colors.shape[0]//2 + 1:] = np.array([0.5, 0.1, 0.1])
sim_viewer.colors = weld_colors
sim_viewer.view(width=500, height=500)

In [ ]:
sim_data = np.genfromtxt("tensile_log.txt", names=True)
lj_energy = sim_data["mdpairLJenergy"]
time_step = sim_data["hoomd_organicslibrarysimulationstensileTensiletimestep"]
temp = sim_data["mdcomputeThermodynamicQuantitieskinetic_temperature"]
plt.plot(temp)
#plt.plot(time_step, lj_energy, linewidth=3)
plt.ylabel("$\dfrac{\epsilon}{N}$", fontsize=15)
plt.xlabel("Time step", fontsize=15)